In [82]:
from lexmachina import LexMachinaAsyncClient
from lexmachina import DistrictCaseQueryRequest
import asyncio
from typing import Optional
from collections import defaultdict

In [69]:
client = LexMachinaAsyncClient(config_file_path='../config/config.ini')

Define three methods to help us retrieve the customized information:
- a method to retrieve all cases from the API in which a given party appeared (perhaps in a given role) within a date range
- a method to retieve the information we want from each of the returned case infos
- a method to call the others and return the result

In [134]:
async def cases_for_party_role(start: str, end: str, party_id: int, role: Optional[str] = None):
    # Retrieve cases for a given party filed within a given date range.  
    # If specified limit the cases to only those in which the party was in a specific role.
    
    query = (
        DistrictCaseQueryRequest()
        .set_date(start, 'filed', 'onOrAfter')
        .set_date(end, 'filed', 'onOrBefore')
    )
    # Filter by role of the party, if specified
    if not role:
        query.include_parties(party_id)
    elif role.lower() == 'defendant':
        query.parties_include_defendant(party_id)
    elif role.lower() == 'plaintiff':
        query.parties_include_plaintiff(party_id)
    # return results as they come
    return await client.query_district_case(query=query, options={'pageThrough': True}, page_size=100)
    
    
def get_plaintiffs_attys_firms_and_judges(c):
    # Retrieve the information of interest
    
    judges = c['judges']
    
    plaintiffs = [p for p in c['parties'] if p['role'] == 'Plaintiff']
    plaintiff_ids = [p['partyId'] for p in plaintiffs]
    
    lawfirms = c['lawFirms']
    plaintiff_firms = [
        lf for lf in lawfirms 
        if all(party_id in plaintiff_ids for party_id in lf['clientPartyIds'])
    ]
    
    attys = c['attorneys']
    plaintiff_attys = [
        atty for atty in attys 
        if all(party_id in plaintiff_ids for party_id in atty['clientsPartyIds'])
    ]
    
    return plaintiffs, plaintiff_firms, plaintiff_attys, judges


async def plaintiff_centric_info_for_date_range(start: str, end: str, party_id: int, role: Optional[str] = None):
    
    # Get the case ids for all district cases that match the params
    case_infos = await cases_for_party_role(start, end, party_id, role)
    case_ids = list(c['districtCaseId'] for c in case_infos)
    print(f'there were {len(case_ids)} cases filed {start} and {end}')
    
    # Gather the case details for each in parallel, extract desired info
    cases = await asyncio.gather(*[client.get_district_cases(case) for case in case_ids])
    return [
        (c['districtCaseId'], *get_plaintiffs_attys_firms_and_judges(c))
        for c in cases
    ]   

Define a class for aggregating the data of interest.  Here we are aggregating by Law Firm, but of course that can be adjusted to suit different information needs.

In [135]:
from collections import Counter

class AggregatedFirmInfo():
    def __init__(self, firm_id: int = None):
        self.firm_id = firm_id
        self.names = Counter()
        self.num_cases = 0
        self.attys = Counter()
        self.parties = Counter()
    
    def name(self):
        return self.names.most_common(1)[0][0]
    
    def __str__(self):
        name_str = '\n\t\t'.join(str(n) for n in self.names.most_common())
        atty_str = '\n\t\t'.join(str(a) for a in self.attys.most_common())
        parties_str = '\n\t\t'.join(str(p) for p in self.parties.most_common())
        return f'''
        AggregatedFirmInfo({self.firm_id}): {self.name()}
         - num cases in subset: {self.num_cases}
         - attys on those cases:\n\t\t{atty_str}
         - parties represented in those cases:\n\t\t{parties_str}
        '''

In [136]:
start_date = '2023-05-01'
end_date = str(datetime.date.today())
liberty_mutual = 39480
role='Defendant'

firm_data = defaultdict(AggregatedFirmInfo)

print(f"Plaintiff Attorney Activity since {start_date}:")
for c, plaintiff_parties, plaintiff_lfs, plaintiff_attys, judges in await plaintiff_centric_info_for_date_range(start_date, end_date, liberty_mutual, role):
    party_name_lut = {
        p['partyId']: p['name']
        for p in plaintiff_parties
    }
    
    for lf in plaintiff_lfs:
        print(c, plaintiff_lfs)
        print()
        
        firm_id = lf['lawFirmId']
        firm_data[firm_id].firm_id = firm_id
        firm_data[firm_id].names[lf['name']] += 1
        # Add to the case tally
        firm_data[firm_id].num_cases += 1
        # Add tallies for the parties represented, here aggregated by name.
        # Of course this can be done by id and converted later to prevent
        # unintentional party merging.
        for client_id in lf['clientPartyIds']:
            party_name = party_name_lut[client_id]
            firm_data[firm_id].parties[party_name] += 1
        
        
    for atty in plaintiff_attys:
        atty_name = atty['name']
        for firm_id in atty['lawFirmIds']:
            # Add tallies for the parties represented, here again, aggregated by name.
            if firm_id in firm_data:
                firm_data[firm_id].attys[atty_name] += 1

Plaintiff Attorney Activity since 2023-05-01:
{'cases': [{'url': 'https://api.lexmachina.com/district-cases/2009041164', 'districtCaseId': 2009041164}, {'url': 'https://api.lexmachina.com/district-cases/2009044340', 'districtCaseId': 2009044340}, {'url': 'https://api.lexmachina.com/district-cases/2009050224', 'districtCaseId': 2009050224}, {'url': 'https://api.lexmachina.com/district-cases/2009063331', 'districtCaseId': 2009063331}, {'url': 'https://api.lexmachina.com/district-cases/2009066535', 'districtCaseId': 2009066535}, {'url': 'https://api.lexmachina.com/district-cases/2009079130', 'districtCaseId': 2009079130}, {'url': 'https://api.lexmachina.com/district-cases/2009083198', 'districtCaseId': 2009083198}, {'url': 'https://api.lexmachina.com/district-cases/2009089588', 'districtCaseId': 2009089588}, {'url': 'https://api.lexmachina.com/district-cases/2009090043', 'districtCaseId': 2009090043}, {'url': 'https://api.lexmachina.com/district-cases/2009090062', 'districtCaseId': 200909

{'cases': [{'url': 'https://api.lexmachina.com/district-cases/2009826206', 'districtCaseId': 2009826206}, {'url': 'https://api.lexmachina.com/district-cases/2009826292', 'districtCaseId': 2009826292}, {'url': 'https://api.lexmachina.com/district-cases/2009826588', 'districtCaseId': 2009826588}, {'url': 'https://api.lexmachina.com/district-cases/2009826651', 'districtCaseId': 2009826651}, {'url': 'https://api.lexmachina.com/district-cases/2009826701', 'districtCaseId': 2009826701}, {'url': 'https://api.lexmachina.com/district-cases/2009826729', 'districtCaseId': 2009826729}, {'url': 'https://api.lexmachina.com/district-cases/2009826730', 'districtCaseId': 2009826730}, {'url': 'https://api.lexmachina.com/district-cases/2009827665', 'districtCaseId': 2009827665}, {'url': 'https://api.lexmachina.com/district-cases/2009830091', 'districtCaseId': 2009830091}, {'url': 'https://api.lexmachina.com/district-cases/2009830306', 'districtCaseId': 2009830306}, {'url': 'https://api.lexmachina.com/dis

In [139]:
for firm_id, info in sorted(firm_data.items(), key=lambda tup: tup[1].num_cases, reverse=True):
    print(info)


        AggregatedFirmInfo(149043834): Cantrell Law Group
         - num cases in subset: 19
         - attys on those cases:
		('Harry E. Cantrell Jr.', 19)
         - parties represented in those cases:
		('Derek Brumfield', 1)
		('James Stewart', 1)
		('Sabra Banks', 1)
		('Betty Brown', 1)
		('John Hurst', 1)
		('Taurie Hoover', 1)
		('Renate Jones', 1)
		('Dianne Chesnut', 1)
		('Frank Green', 1)
		('Meah Badeaux', 1)
		('Darian Webb Gordon', 1)
		('Dewitt Madison', 1)
		('Matt Fiske', 1)
		('Tony Sanders', 1)
		('Bonnie Viveiros', 1)
		('Ronnie Martin', 1)
		('Michael A Marque', 1)
		('Betty McCaleb', 1)
		('Corey Major', 1)
        

        AggregatedFirmInfo(185234158): Galindo Law Firm
         - num cases in subset: 14
         - attys on those cases:
		('Mark Richard Ladd', 7)
		('Chinwe Onyenekwu', 7)
         - parties represented in those cases:
		('James Stewart', 1)
		('John Hurst', 1)
		('Taurie Hoover', 1)
		('Renate Jones', 1)
		('Frank Green', 1)
		('Meah Badeaux'